In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# link do repozytorium: https://www.kaggle.com/datasets/kazanova/sentiment140

#dodaj właściwe kolumny zgodnie z dokumentacją oraz zdefiniuj plik z danymi w ramce pandas
columns = []
data = pd.read_csv("", names=columns)

# Print the shape of dataframe
print(data.shape)

# Print top 5 rows
data.head(5)




FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv'

In [ ]:
# Przekształć etykiety „sentiment” na wartości binarne
data['sentiment'] = data['sentiment'].apply(lambda x: 1 if x == "positive" else 0)

# Podziel na zbiory treningowy i testowy
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

# Tokenizacja tekstu
max_words = 10000
maxlen = 100  # Długość tweetów

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_data['text'])

# Zamień tekst na sekwencje liczbowe
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data['text']), maxlen=maxlen)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data['text']), maxlen=maxlen)
y_train = train_data['sentiment'].values
y_test = test_data['sentiment'].values

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=maxlen),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')  # Wyjście binarne
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
batch_size = 64
epochs = 5

history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    verbose=1
)


In [ ]:
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print(f"Test Accuracy: {acc}")


In [ ]:
def predict_sentiment(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=maxlen)
    prediction = model.predict(padded)
    return "Positive" if prediction > 0.5 else "Negative"

# Przykład prognozy
print(predict_sentiment("I love this product!"))
